# Bim File

variant identifier and minor allele

In [25]:
import pandas as pd
df_bim=pd.read_csv('Data/0_Raw/DataS1.bim', sep='\t', header=None)
# df_bim = 1 and 4 columns
df_bim = df_bim[[1, 4]]

df_bim.rename(columns={1: 'VariantID', 4: 'minor'}, inplace=True)

In [26]:
# conver df_bim['variantID'] to a list
columns= df_bim['VariantID'].to_list()
columns.insert(0, 'familyID')
columns.insert(1, 'individualID')
columns.insert(2, 'fatherID')
columns.insert(3, 'motherID')
columns.insert(4,'Sex')
columns.insert(5, 'date')

In [27]:
minor=df_bim['minor'].to_list()
minor.insert(0, 'None')
minor.insert(1, 'None')
minor.insert(2, 'None')
minor.insert(3, 'None')
minor.insert(4,'None')
minor.insert(5, 'None')

# Ped File

familly id and individual id and allels

## Split the ped file by familly id

In [28]:
! rm -r Data/1_raw2freq/countries/*

rm: cannot remove 'Data/1_raw2freq/countries/*': No such file or directory


In [29]:
import os
 
def split_and_convert(file_path):
    # Create a directory to store output files
    os.makedirs('Data/1_raw2freq/countries', exist_ok=True)
    populations = []
    with open(file_path, 'r') as ped_file:
        for row in ped_file:
            row_data = row.strip().split()
            populations.append(row_data[0])
            pedigree = row_data[0]
            with open(f'Data/1_raw2freq/countries/{pedigree}.txt', 'a') as out_file:
                # Join the pairs of alleles in each row
                new_fields = row_data[:6]
                for i in range(6, len(row_data), 2):
                    new_fields.append(row_data[i] + row_data[i+1])
                # Write the new fields to the output file
                out_file.write(' '.join(new_fields) + '\n')
        
    return list(set(populations))





file_path = 'Data/0_Raw/DataS1.ped'

populations = split_and_convert(file_path)
print(populations)



['UnitedKingdom', 'CzechRepublic', 'Germany', 'Spain', 'Lithuania', 'Luxembourg', 'Austria', 'Hungary', 'Estonia', 'Georgia', 'Norway', 'Turkey', 'Portugal', 'Macedonia', 'Switzerland', 'Latvia', 'Russia', 'Jordan', 'Ukraine', 'Iran', 'Israel', 'Poland', 'Armenia', 'Serbia', 'Greece', 'Bulgaria', 'Italy', 'France', 'TheNetherlands', 'Denmark', 'Kazakhstan', 'Croatia', 'Romania', 'Sweden']


In [30]:
dfeuro = pd.read_excel(
    'Data/0_Raw/Eurasian - Dataset_tims.xlsx', sheet_name='Eurasian')
dfeuro = pd.DataFrame(dfeuro)
dfeuro.rename(columns={
              'Date mean in BP in years before 1950 CE [OxCal mu for a direct radiocarbon date, and average of range for a contextual date]': 'date'}, inplace=True)

# date_range is a list of the range of dates from 0 to 12000 years ago in 1000 year intervals
date_range = list(range(0, 12000, 1000)) 
# if dfeuro['date'] is in the range of date_range, then dfeuro['cat_date'] is the index of the range
dfeuro['cat_date']=pd.cut(dfeuro['date'], date_range, labels=range(0, len(date_range)-1))
# multiply the index by 1000 to get the range of dates
dfeuro['cat_date']=dfeuro['cat_date'].astype(int)*1000

dfeuro['cat_date']

0       5000
1       5000
2       4000
3       5000
4       5000
        ... 
4153    9000
4154    1000
4155    7000
4156    9000
4157    4000
Name: cat_date, Length: 4158, dtype: int64

In [31]:

def freq(pop_path, country):
    '''This function takes a population file and returns a dictionary of the frequency of each variant'''
    df = pd.read_csv(pop_path, sep=' ', header=None)
    df.columns = columns
    df.date = 'N'
    # print(len(df))
    # locate df['individualID'] in dfeuro['Master ID'] and add df['cat_date'] to the df[date]
    for i in range(0, len(df)):
        for j in range(0, len(dfeuro)):
            if df['individualID'][i] == dfeuro['Master ID'][j]:
                df['date'][i] = dfeuro['cat_date'][j]
  
    # get unique dates in df['date'] and store in a list and remove None
    date = df['date'].unique()
    date = date.tolist()
    if 'N' in date:
        date.remove('N')
    

    countryfreq = {}
    
    for d in date:
        freqdate= []
        df2 = df[df['date'] == d]
        for i in range(6, len(df.columns)):
            minorAllele = minor[i]
            count = 0
            total = len(df2)*2
            for j in range(0, len(df2)):
                # count occurrences of the minor allele in column i
                count += str(df2.iloc[j, i]).count(str(minorAllele))
            freqdate.append(count/total)
            
        name= f'{country}_{d}'
        countryfreq[name] = freqdate
        
    return countryfreq





In [ ]:
import multiprocessing
! rm Data/S1_freq.csv
def process_country(country):
    pop_path = f'Data/1_raw2freq/countries/{country}.txt'
    freq_results = freq(pop_path, country)
    with open('Data/1_raw2freq/S1_freq.csv', 'a') as f:
        for key, value in freq_results.items():
            countr = key.split('_')[0]
            datee= key.split('_')[1]
            str_values = [str(val) for val in value]  # convert float values to strings
            f.write('%s\t%s\t%s\n' % (countr, datee,'\t'.join(str_values)))

if __name__ == '__main__':
    with multiprocessing.Pool() as pool:
        pool.map(process_country, populations)

## Filter SNPs which have 0 in more than 80% of the countries-age groups

In [34]:
df = pd.read_csv('Data/1_raw2freq/S1_freq.csv', sep='\t', header=None)
#firts coumn name is country and from second to last column name is columns[5:]
df.columns = ['country'] + columns[5:]


,country,date,rs3094315,rs6696609,rs9442372,rs4970405,rs11807848,rs4970421,rs2887286,rs113791678,...,rs739365,rs148828263,rs9616945,rs5770820,rs6010062,rs715586,rs8137951,rs3810648,rs78827609,rs116026001
0,Austria,7000,0.000000,0.0,0.5,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,Austria,6000,0.166667,0.5,1.0,0.0,0.0,0.0,0.0,0.0,...,0.166667,0.0,0.166667,0.166667,0.333333,0.0,0.5,0.0,0.0,0.0


In [16]:
dff=[]
for i in range(0, len(df.columns)):
   # count the number of times a value == 0 in each column
    count = df[i].value_counts().get(0)
    dff.append(count)

In [17]:
df2=df.copy()
# get indeces of dff where that are > 80
index = [i for i, x in enumerate(dff) if x > 95]
# remove columns 0 and 1 from index
index.remove(0)
index.remove(1)
# remove df2 columns that are in index
df2.drop(df2.columns[index], axis=1, inplace=True)
df2.shape

(119, 1058)

In [18]:
# save to csv without index and header
# df2.to_csv('Data/1_raw2freq/S1_freq_filtered.csv', index=False, header=False, sep='\t')

df2.to_csv('Data/1_raw2freq/test.csv', index=False, header=False, sep='\t')


In [19]:
# convert every *.csv file in the directory to *.csv.gz
! gzip -9 Data/1_raw2freq/*.csv
